# Phase 5 — Overlay and Metrics

Visualize navigation state on video frames and collect performance metrics.

## Imports and setup

In [ ]:
from pathlib import Path
import sys
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.utils import (
    load_config,
    ensure_demo_video,
    open_capture,
    iter_frames,
    release_capture,
    draw_navigation_overlay,
)
from src.perception_flow import compute_optical_flow, compute_obstacle_risk, compute_openness
from src.memory import NavigationMemory
from src.fsm import NavigationFSM
from src.controller import NavigationController
from src.metrics import NavigationMetrics

config = load_config("configs/default.yaml")
video_path = ensure_demo_video(config)
print(f"Video: {video_path}")

## Full navigation loop with overlay and metrics

In [ ]:
cap = open_capture(config)
max_frames = config["video"].get("max_frames") or 200

# Initialize components
memory = NavigationMemory(config)
fsm = NavigationFSM(config)
controller = NavigationController(config)
metrics = NavigationMetrics()

# Start metrics
metrics.start()

# Storage for visualization
overlay_frames = []
frame_indices = []

prev_gray = None
process_count = 0

for idx, frame in tqdm(iter_frames(cap, max_frames=max_frames, convert_rgb=False), total=max_frames):
    frame_start = time.perf_counter()
    
    curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prev_gray is not None:
        # Perception
        flow = compute_optical_flow(prev_gray, curr_gray, config)
        risk_raw = compute_obstacle_risk(flow, config)
        openness_raw, direction = compute_openness(flow, config)
        
        # Memory
        signals = memory.update(risk_raw, openness_raw, direction)
        
        # FSM
        state = fsm.update(signals)
        
        # Controller
        recovery_dir = fsm.get_recovery_direction()
        preferred_dir = recovery_dir if recovery_dir else signals["preferred_direction"]
        command = controller.get_command(state, preferred_dir)
        
        # Compute latency
        frame_end = time.perf_counter()
        latency = frame_end - frame_start
        
        # Record metrics
        metrics.record(state, latency)
        
        # Draw overlay
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        fps = metrics.perf.get_fps()
        frame_overlay = draw_navigation_overlay(
            frame_rgb,
            state=state,
            command_name=command.command_name,
            risk=signals["risk_smooth"],
            openness=signals["openness_smooth"],
            fps=fps,
            latency_ms=latency * 1000,
        )
        
        # Store sample frames for visualization
        if idx % 20 == 0:  # Sample every 20 frames
            overlay_frames.append(frame_overlay)
            frame_indices.append(idx)
        
        process_count += 1
    
    prev_gray = curr_gray

release_capture(cap)

print(f"\nProcessed {process_count} frames")

## Display sample frames with overlay

In [ ]:
# Display a grid of sample frames
num_samples = min(6, len(overlay_frames))
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i in range(num_samples):
    axes[i].imshow(overlay_frames[i])
    axes[i].set_title(f"Frame {frame_indices[i]}")
    axes[i].axis("off")

# Hide unused subplots
for i in range(num_samples, 6):
    axes[i].axis("off")

plt.suptitle("Sample Frames with Navigation Overlay")
plt.tight_layout()
plt.show()

## Print metrics summary

In [ ]:
metrics.print_summary()

## Visualize metrics over time

In [ ]:
# Compute rolling FPS
frame_times = metrics.perf.frame_times
rolling_fps = []
window = 30

for i in range(window, len(frame_times)):
    elapsed = frame_times[i] - frame_times[i - window]
    fps = window / elapsed if elapsed > 0 else 0
    rolling_fps.append(fps)

# Latencies
latencies_ms = np.array(metrics.perf.latencies) * 1000

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Rolling FPS
axes[0].plot(range(window, len(frame_times)), rolling_fps, label="Rolling FPS (30 frames)")
axes[0].axhline(metrics.perf.get_fps(), color="red", linestyle="--", label=f"Average FPS: {metrics.perf.get_fps():.1f}")
axes[0].set_ylabel("FPS")
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_title("Performance Over Time")

# Frame latency
axes[1].plot(latencies_ms, label="Frame Latency", alpha=0.6)
latency_stats = metrics.perf.get_latency_stats()
axes[1].axhline(latency_stats["mean"], color="orange", linestyle="--", label=f"Mean: {latency_stats['mean']:.1f} ms")
axes[1].axhline(latency_stats["p95"], color="red", linestyle="--", label=f"P95: {latency_stats['p95']:.1f} ms")
axes[1].set_ylabel("Latency (ms)")
axes[1].set_xlabel("Frame")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## State duration histogram

In [ ]:
state_percentages = metrics.states.get_state_percentages()

if state_percentages:
    states = list(state_percentages.keys())
    percentages = list(state_percentages.values())
    
    plt.figure(figsize=(10, 6))
    plt.bar(states, percentages, color=["green", "orange", "orange", "red", "purple"][:len(states)])
    plt.ylabel("Percentage of Time (%)")
    plt.xlabel("State")
    plt.title("Time Spent in Each Navigation State")
    plt.xticks(rotation=45, ha="right")
    plt.grid(True, alpha=0.3, axis="y")
    plt.tight_layout()
    plt.show()
else:
    print("No state data available")

## Verification

Phase 5 verification:
- Overlays display state, command, risk, and openness on frames
- FPS and latency are measured and displayed
- End-of-run summary shows state distribution and performance metrics
- Metrics indicate stable, real-time capable performance